<a href="https://colab.research.google.com/github/Deepak-Mewada/NeuralDecoder/blob/main/BrainDecoder/Basics/Simple_Training_on_mne_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install braindecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00


ImportError: ignored

In [ ]:
from braindecode.models import ShallowFBCSPNet

In [ ]:
print(ShallowFBCSPNet.__doc__)

Shallow ConvNet model from Schirrmeister et al 2017.

Model described in [Schirrmeister2017]_.

Parameters
----------
n_chans : int
    Number of EEG channels.
n_outputs : int
    Number of outputs of the model. This is the number of classes
    in the case of classification.
n_times : int
    Number of time samples of the input window.
n_filters_time: int
    Number of temporal filters.
filter_time_length: int
    Length of the temporal filter.
n_filters_spat: int
    Number of spatial filters.
pool_time_length: int
    Length of temporal pooling filter.
pool_time_stride: int
    Length of stride between temporal pooling filters.
final_conv_length: int | str
    Length of the final convolution layer.
    If set to "auto", length of the input signal must be specified.
conv_nonlin: callable
    Non-linear function to be used after convolution layers.
pool_mode: str
    Method to use on pooling layers. "max" or "mean".
pool_nonlin: callable
    Non-linear function to be used after poolin

In [ ]:
model = ShallowFBCSPNet(n_chans = 32, n_times = 1000, n_outputs = 2, final_conv_length = 'auto')

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [ ]:
print(model)


Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 32, 1000]             [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 32, 1000]             [1, 32, 1000, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 32, 1000, 1]          [1, 1, 1000, 32]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1000, 32]          [1, 40, 976, 1]           52,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 976, 1]           [1, 40, 976, 1]           80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 976, 1]           [1, 40, 976, 1]           --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 976, 1]           [1, 40, 61, 1]            --                        [75, 1]
├─Express

In [ ]:
import mne
import numpy as np

info = mne.create_info(ch_names = ['C3', 'C4', 'Cz'], sfreq = 256., ch_types = 'eeg')
X = np.random.randn(100, 3, 1024)
epochs = mne.EpochsArray(X, info = info)
y = np.random.randint(0, 4, size = 100)
print(epochs)

Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
<EpochsArray |  100 events (all good), 0 – 3.99609 s, baseline off, ~2.4 MB, data loaded,
 '1': 100>


In [ ]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

net = EEGClassifier('ShallowFBCSPNet', module__final_conv_length = 'auto',
                    train_split = ValidSplit(0.2))

In [ ]:
net.fit(epochs, y)

  epoch    valid_acc    valid_loss     dur
-------  -----------  ------------  ------
      1       0.4000       22.1031  0.0511
      2       0.4000       22.1031  0.0205
      3       0.4000       22.1031  0.0194
      4       0.4000       22.1031  0.0199
      5       0.4000       22.1031  0.0201
      6       0.4000       22.1031  0.0194
      7       0.4000       22.1031  0.0192


/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


      8       0.4000       22.1031  0.0202
      9       0.4000       22.1031  0.0191
     10       0.4000       22.1031  0.0189


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 3, 1024]              [1, 4]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 3, 1024]              [1, 3, 1024, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 3, 1024, 1]           [1, 1, 1024, 3]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1024, 3]           [1, 40, 1000, 1]          5,840                     --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1000, 1]          [1, 40, 1000, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

In [ ]:
print(net.module_)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 3, 1024]              [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 3, 1024]              [1, 3, 1024, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 3, 1024, 1]           [1, 1, 1024, 3]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1024, 3]           [1, 40, 1000, 1]          5,840                     --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1000, 1]          [1, 40, 1000, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1000, 1]          [1, 40, 1000, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1000, 1]          [1, 40, 62, 1]            --                        [75, 1]
├─Express

In [ ]:
print(f'{net.module_.n_chans = }\n{net.module_.n_times=}\n{net.module_.n_outputs=}'
      f'\n{net.module_.input_window_seconds=}\n{net.module_.sfreq=}\n{net.module_.chs_info=}')